<a href="https://colab.research.google.com/github/rishipython/ALL-Cell-Classification/blob/main/LeukemiaClassificationTraining2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121
from keras.layers import Add
from keras.layers import Dense
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Activation, BatchNormalization, PReLU, Dropout, Conv2DTranspose
from keras.layers import Activation
from keras.optimizers import Adam
from keras.layers.merge import concatenate
from keras.utils import to_categorical
import PIL
import numpy as np
from PIL import Image
from numpy import asarray
import glob
import nibabel as nib
import os
import tarfile as trf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.metrics import roc_auc_score, roc_curve, auc
from IPython.display import display
K.set_image_data_format("channels_last")

In [ ]:
# Access GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Apr 11 21:20:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Access more memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
# See if Tensorflow is using GPU
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_dir = f"{os.sep}content{os.sep}drive{os.sep}MyDrive{os.sep}leukemia_classification{os.sep}"

# Model Construction

In [ ]:
def make_model(input_shape=(450, 450, 3), lr=1e-4, loss_function="binary_crossentropy", metrics="accuracy"):
  if not isinstance(metrics, list):
    metrics = [metrics]
  x = Input(input_shape)
  densenet = DenseNet121(input_shape=input_shape, include_top=False)(x)
  vgg16 = VGG16(input_shape=input_shape, include_top=False)(x)
  resnet50 = ResNet50(input_shape=input_shape, include_top=False)(x)
  conv1 = Conv2D(3, vgg16.shape[1]-9)(vgg16)
  conv2 = Conv2D(3, resnet50.shape[1]-9)(resnet50)
  add1 = Add()([conv1, conv2])
  up = UpSampling2D(size=(10, 10))(add1)
  inceptionv3 = InceptionV3(input_shape=up.shape[1:], include_top=False)(up)
  conv3 = Conv2D(1, densenet.shape[1])(densenet)
  conv4 = Conv2D(1, 1)(inceptionv3)
  add2 = Add()([conv3, conv4])
  output_layer = Activation("sigmoid")(add2)
  model = Model(x, output_layer)
  model.compile(optimizer = Adam(lr = lr), loss = loss_function, metrics = metrics)
  return model

# Make and train model_4

In [ ]:
x_train = np.load(drive_dir+"x_train.npy")
y_train = np.load(drive_dir+"y_train.npy")
x_val = np.load(drive_dir+"x_val.npy")
y_val = np.load(drive_dir+"y_val.npy")
x_test = np.load(drive_dir+"x_test.npy")
y_test = np.load(drive_dir+"y_test.npy")

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(1020, 450, 450, 3)
(1020, 1, 1, 1)
(340, 450, 450, 3)
(340, 1, 1, 1)
(340, 450, 450, 3)
(340, 1, 1, 1)


In [ ]:
mean = float(open(drive_dir+"mean.txt", 'r').readline())
std = float(open(drive_dir+"std.txt", 'r').readline())

In [ ]:
print(mean)
print(type(mean))
print(std)
print(type(std))

9.178323056564189
<class 'float'>
23.645296284294893
<class 'float'>


In [ ]:
input_shape = x_train[0].shape
output_shape = (1, 1, 1)

In [ ]:
print(input_shape)
print(output_shape)

(450, 450, 3)
(1, 1, 1)


In [ ]:
lr = 1e-4
loss_function = "binary_crossentropy"
metrics = "accuracy"
if not isinstance(metrics, list):
  metrics = [metrics]

In [ ]:
# Model 4 (proposed model)
model_4 = make_model()

87916544/87910968 [==============================] - 0s 0us/step


In [ ]:
model_4.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 450, 450, 3) 0                                            
__________________________________________________________________________________________________
vgg16 (Functional)              (None, 14, 14, 512)  14714688    input_1[0][0]                    
__________________________________________________________________________________________________
resnet50 (Functional)           (None, 15, 15, 2048) 23587712    input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 10, 10, 3)    38403       vgg16[0][0]                      
______________________________________________________________________________________________

In [ ]:
epochs = 33
batch_size = 10

In [ ]:
# Tran model
history = model_4.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=batch_size)
if input("Press Y to save: ") == "Y":
  model_4.save(drive_dir + 'model_4')
  hist_df = pd.DataFrame(history.history)
  hist_csv_file = drive_dir + 'history_4.csv'
  with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

Epoch 1/33
102/102 [==============================] - 138s 912ms/step - loss: 1.0504 - accuracy: 0.6932 - val_loss: 0.6121 - val_accuracy: 0.7529
Epoch 2/33
102/102 [==============================] - 88s 864ms/step - loss: 0.5088 - accuracy: 0.8411 - val_loss: 16.5310 - val_accuracy: 0.5853
Epoch 3/33
102/102 [==============================] - 88s 863ms/step - loss: 0.8315 - accuracy: 0.8114 - val_loss: 0.4285 - val_accuracy: 0.8471
Epoch 4/33
102/102 [==============================] - 88s 863ms/step - loss: 0.3793 - accuracy: 0.8886 - val_loss: 83.0779 - val_accuracy: 0.4324
Epoch 5/33
102/102 [==============================] - 88s 863ms/step - loss: 0.2776 - accuracy: 0.8941 - val_loss: 34.9124 - val_accuracy: 0.5706
Epoch 6/33
102/102 [==============================] - 88s 863ms/step - loss: 0.2439 - accuracy: 0.9143 - val_loss: 5.3984 - val_accuracy: 0.5382
Epoch 7/33
102/102 [==============================] - 88s 863ms/step - loss: 0.0853 - accuracy: 0.9704 - val_loss: 11.6178 - v